# Imports

In [ ]:
import pandas as pd # 1.1
from sklearn.preprocessing import StandardScaler # 1.2
from sklearn.model_selection import train_test_split # 1.4

# 1. Data Preprocessing

## 1.1 Get Data

## 1.2 Z-Score Normalization / Standardization

In [ ]:
# Select the columns with player stats
stats_columns = ['points', 'rebounds', 'assists', 'steals', 'blocks']  # Make sure we got all the columns here

# Initialize the scaler
scaler = StandardScaler()

# Normalize the stats using Z-score normalization
df[stats_columns] = scaler.fit_transform(df[stats_columns])


"\n# Select the columns with player stats\nstats_columns = ['points', 'rebounds', 'assists', 'steals', 'blocks']  # Make sure we got all the columns here\n\n# Initialize the scaler\nscaler = StandardScaler()\n\n# Normalize the stats using Z-score normalization\ndf[stats_columns] = scaler.fit_transform(df[stats_columns])\n"

## 1.3 One-Hot Encoding

In [ ]:
# One-hot encode the categorical columns
df = pd.get_dummies(df, columns=['opp_team'])

## 1.4 Train-Test Split

In [ ]:
# Split the data
X = df.drop(['fp'], axis=1)  # Features
y = df['fp']  # Target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=6)